In [1]:
# Translated to .py by Meritxell Pacheco
# 2017
# Adapted to PandasBiogeme by Michel Bierlaire
# Sun Oct 21 23:15:31 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit

pandas = pd.read_table("lpmc02.dat")
database = db.Database("LPMC",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

# Exclude
# exclude = (  ArrivalTimeHours_1   ==  -1  )
# database.remove(exclude)
  
# Choice
chosenAlternative = travel_mode

#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1	 = Beta('Constant1',0,None,None,1)
Constant2	 = Beta('Constant2',0,None,None,0)
Constant3	 = Beta('Constant3',0,None,None,0)
Constant4	 = Beta('Constant4',0,None,None,0)
Cost	 = Beta('Cost',0,None,None,0)
Total_TT1	 = Beta('Total_TT1',0,None,None,0)
Total_TT2	 = Beta('Total_TT2',0,None,None,0)
Total_TT3	 = Beta('Total_TT3',0,None,None,0)
Total_TT4	 = Beta('Total_TT4',0,None,None,0)
TrainCard	 = Beta('TrainCard',0,None,None,0)
BusCard	 = Beta('BusCard',0,None,None,0)

# Define here arithmetic expressions for name that are not directly
# available from the data

dur_public = DefineVariable('dur_public', dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int ,database)
cost_public = DefineVariable('cost_public', cost_transit ,database)
cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving

Opt1 = Constant1 + Total_TT1 * dur_walking
Opt2 = Constant2 + Total_TT2 * dur_cycling
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * dur_public + TrainCard * faretype + BusCard * bus_scale
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * dur_driving
V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1,2: 1,3: 1,4: 1}

# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_lpmc_model2_PtCards"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

            Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
BusCard    -0.553    0.137   -4.04 5.36e-05          0.14        -3.94   
Constant2   -4.61    0.173   -26.7        0         0.215        -21.4   
Constant3   -1.74    0.198   -8.77        0         0.221        -7.84   
Constant4   -1.88    0.107   -17.5        0          0.15        -12.5   
Cost       -0.138   0.0154   -8.97        0        0.0143        -9.64   
Total_TT1   -8.22    0.293     -28        0         0.455        -18.1   
Total_TT2   -5.96    0.446   -13.4        0         0.514        -11.6   
Total_TT3   -3.69    0.238   -15.5        0         0.245        -15.1   
Total_TT4   -6.73    0.341   -19.7        0         0.386        -17.4   
TrainCard -0.0926   0.0367   -2.52   0.0116        0.0377        -2.46   

           Rob. p-value  
BusCard         8.1e-05  
Constant2             0  
Constant3      4.44e-15  
Constant4             0  
Cost                  0  
Total_TT1             0  
Tot